<a href="https://colab.research.google.com/github/sweetbaer03/EN_Sentiment-Analysis/blob/main/friends_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FRIENDS Sentiment Analysis**


Bert 모델 사용 

In [1]:
#transformers 설치(colab 사용시)
!pip install transformers

     |████████████████████████████████| 1.5MB 5.6MB/s 
     |████████████████████████████████| 890kB 20.8MB/s 
     |████████████████████████████████| 2.9MB 13.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1d914fab754f1d1ea9ddd31d8d708d5a6538eaba4689843e0c02e9442516b537
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
import time
import datetime
import json
import re
import nltk
import matplotlib.pyplot as plt

from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# 데이터처리

In [3]:
!git clone https://github.com/sweetbaer03/EN_Sentiment-Analysis.git

Cloning into 'EN_Sentiment-Analysis'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 6), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [4]:
MAX_LEN = 85
batch_size = 32
epochs = 20

TOKEN_MODEL = 'bert-base-uncased'
TRAIN_MODEL = 'bert-base-uncased'

#TOKEN_MODEL = 'uncased_L-12_H-768_A-12'
#TRAIN_MODEL = 'uncased_L-12_H-768_A-12'

#TOKEN_MODEL = 'google/electra-large-discriminator'
#TRAIN_MODEL = 'google/electra-large-generator'

## 1.1 프렌즈 / 캐글 테스트 데이터

In [5]:
def jsonToDf(file_name):
  with open(file_name, encoding = 'utf-8', mode = 'r') as file:
    json_array = json.load(file)
  
  result = pd.DataFrame.from_dict(json_array[0])

  is_first = True
  for array in json_array:
    if is_first:
      is_first = False
      continue
    
    temp_df = pd.DataFrame.from_dict(array)
    result = result.append(temp_df, ignore_index = True)

  return result

In [6]:
train_data = jsonToDf('EN_Sentiment-Analysis/data_in/friends_train.json')
dev_data   = jsonToDf('EN_Sentiment-Analysis/data_in/friends_dev.json')
test_data  = jsonToDf('EN_Sentiment-Analysis/data_in/friends_test.json')

In [7]:
#합치기 위해 포멧 통일
train_data.drop(labels=['speaker','annotation'], axis="columns", inplace=True)
dev_data.drop(labels=['speaker','annotation'], axis="columns", inplace=True)
test_data.drop(labels=['speaker','annotation'], axis="columns", inplace=True)

In [8]:
emotions = train_data['emotion'].values

**캐글** 테스트 데이터

In [9]:
test_df = pd.read_csv('EN_Sentiment-Analysis/data_in/en_data.csv')#, encoding = 'unicode_escape')

In [10]:
test_df

,id,i_dialog,i_utterance,speaker,utterance
0,0,0,0,Phoebe,"Alright, whadyou do with him?"
1,1,0,1,Monica,Oh! You're awake!
2,2,0,2,Joey,Then you gotta come clean with Ma! This is not...
3,3,0,3,Mr. Tribbiani,"Yeah, but this is"
4,4,0,4,Joey,I don't wanna hear it! Now go to my room!
...,...,...,...,...,...
1618,1618,150,14,Joey,Nooo.
1619,1619,150,15,Lauren,"Hi, Kate!"
1620,1620,150,16,Kate,"Hi, Lauren."
1621,1621,150,17,Joey,"Hi, Lauren."


### 프랜즈 및 캐글테스트 마스크 처리

프랜즈 데이터 마스크

In [11]:
print(train_data.shape)
print(dev_data.shape)
print(test_data.shape)
print(test_df.shape)

(10561, 2)
(1178, 2)
(2764, 2)
(1623, 5)


In [12]:
def cleaning1(str):
    replaceAll= str
    only_english = re.sub('\x92', '\'', replaceAll)
    return only_english


def getInputsAndLabels(dataset):
  data = dataset.copy(deep=True)

  utterances = data['utterance']
  utterances = ["[CLS] " + str(cleaning1(utterance)) + " [SEP]" for utterance in utterances]#이거다!
  
  encoder = LabelEncoder()
  labels = data['emotion'].values
  encoder.fit(labels)
  labels = encoder.transform(labels)

  tokenizer = BertTokenizer.from_pretrained(TOKEN_MODEL, do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, labels, attention_masks


def getInputsFromTest(dataset):
  data = dataset.copy(deep=True)

  utterances = data['utterance']
  #utterances = ["[CLS] " + str(utterance) + " [SEP]" for utterance in utterances]
  utterances = ["[CLS] " + str(cleaning1(utterance)) + " [SEP]" for utterance in utterances]#이거다!
  
  tokenizer = BertTokenizer.from_pretrained(TOKEN_MODEL, do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, attention_masks


def getIndex(dataset):
  data = dataset.copy(deep = True)
  input_index = data.id.tolist()
  return torch.tensor(input_index)

In [13]:
train_inputs1, train_labels1, train_masks1 = getInputsAndLabels(train_data)
dev_inputs, dev_labels, dev_masks = getInputsAndLabels(dev_data)
test_inputs, test_labels, test_masks = getInputsAndLabels(test_data)
test_df_inputs, test_df_masks = getInputsFromTest(test_df)

In [14]:
print('전체 프랜즈 학  습 데이터의 개수: {}'.format(len(train_inputs1)))
print('전체 프랜즈 라  벨 데이터의 개수: {}'.format(len(train_labels1)))
print('전체 프랜즈 마스크 데이터의 개수: {}'.format(len(train_masks1)))

전체 프랜즈 학  습 데이터의 개수: 10561
전체 프랜즈 라  벨 데이터의 개수: 10561
전체 프랜즈 마스크 데이터의 개수: 10561


In [15]:
print('전체 프랜즈 dev학  습 데이터의 개수: {}'.format(len(dev_inputs)))
print('전체 프랜즈 dev라  벨 데이터의 개수: {}'.format(len(dev_labels)))
print('전체 프랜즈 dev마스크 데이터의 개수: {}'.format(len(dev_masks)))

전체 프랜즈 dev학  습 데이터의 개수: 1178
전체 프랜즈 dev라  벨 데이터의 개수: 1178
전체 프랜즈 dev마스크 데이터의 개수: 1178


In [16]:
print('전체 프랜즈 test학  습 데이터의 개수: {}'.format(len(test_inputs)))
print('전체 프랜즈 test라  벨 데이터의 개수: {}'.format(len(test_labels)))
print('전체 프랜즈 test마스크 데이터의 개수: {}'.format(len(test_masks)))

전체 프랜즈 test학  습 데이터의 개수: 2764
전체 프랜즈 test라  벨 데이터의 개수: 2764
전체 프랜즈 test마스크 데이터의 개수: 2764


In [17]:
print('전체 프랜즈 test1 학  습 데이터의 개수: {}'.format(len(test_df_inputs)))
print('전체 프랜즈 test1 마스크 데이터의 개수: {}'.format(len(test_df_masks)))

전체 프랜즈 test1 학  습 데이터의 개수: 1623
전체 프랜즈 test1 마스크 데이터의 개수: 1623


## 1.2 IMDB 데이터

In [22]:
#IMDB data
train_data_db = pd.read_csv('EN_Sentiment-Analysis/data_in/labeledTrainData.tsv', header = 0, delimiter = '\t', quoting = 3)
train_data_db.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [24]:
def cleaning_mv(str):
    replaceAll= str
    replaceAll = re.sub('<br />', '', replaceAll)
    return replaceAll

# 감정을 숫자로 변환
def emotion_labeling_mv(emotion):
   return{0 : 'anger',1:'joy'}[emotion]

emotion_labels = []

for e in train_data_db['sentiment']:
   emotion_labels.append(emotion_labeling_mv(e))

train_data_db['label'] = emotion_labels
train_data_db[:10]

,id,sentiment,review,label
0,"""5814_8""",1,"""With all this stuff going down at the moment ...",joy
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ...",joy
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell...",anger
3,"""3630_4""",0,"""It must be assumed that those who praised thi...",anger
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ...",joy
5,"""8196_8""",1,"""I dont know why people think this is such a b...",joy
6,"""7166_2""",0,"""This movie could have been very good, but com...",anger
7,"""10633_1""",0,"""I watched this video at a friend's house. I'm...",anger
8,"""319_1""",0,"""A friend of mine bought this film for £1, and...",anger
9,"""8713_10""",1,"""<br /><br />This movie is full of references....",joy


### IMDB 데이터 마스크 처리

In [25]:
def getInputsAndLabels2(dataset):
  data = dataset.copy(deep=True)

  utterances = data['review']
  utterances = ["[CLS] " + str(cleaning_mv(utterance)) + " [SEP]" for utterance in utterances]
  
  encoder = LabelEncoder()
  labels = data['label'].values
  encoder.fit(labels)
  labels = encoder.transform(labels)

  tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
  tokenized_texts = [tokenizer.tokenize(utterance) for utterance in utterances]

  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  attention_masks = []
  for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

  return input_ids, labels, attention_masks

In [26]:
train_inputs2, train_labels2, train_masks2 = getInputsAndLabels2(train_data_db)

In [27]:
print('전체 IMDB 학  습 데이터의 개수: {}'.format(len(train_inputs2)))
print('전체 IMDB 라  벨 데이터의 개수: {}'.format(len(train_labels2)))
print('전체 IMDB 마스크 데이터의 개수: {}'.format(len(train_masks2)))

전체 IMDB 학  습 데이터의 개수: 25000
전체 IMDB 라  벨 데이터의 개수: 25000
전체 IMDB 마스크 데이터의 개수: 25000


## 1.3 훈련 데이터 합치기

In [18]:
train_masks0 = []
train_masks0.extend(train_masks1)
#train_masks0.extend(train_masks2)
#train_masks0.extend(train_masks3)

In [19]:
train_labels0 = []
train_labels0.extend(train_labels1)
#train_labels0.extend(train_labels2)
#train_labels0.extend(train_labels3)

In [20]:
train_inputs0 = []
train_inputs0.extend(train_inputs1)
#train_inputs0.extend(train_inputs2)
#train_inputs0.extend(train_inputs3)

In [21]:
print('전체 학  습 데이터의 개수: {}'.format(len(train_inputs0)))
print('전체 라  벨 데이터의 개수: {}'.format(len(train_labels0)))
print('전체 마스크 데이터의 개수: {}'.format(len(train_masks0)))

전체 학  습 데이터의 개수: 10561
전체 라  벨 데이터의 개수: 10561
전체 마스크 데이터의 개수: 10561


In [22]:
train_inputs = torch.tensor(train_inputs0)
train_labels = torch.tensor(train_labels0)
train_masks = torch.tensor(train_masks0)

dev_inputs = torch.tensor(dev_inputs)
dev_labels = torch.tensor(dev_labels)
dev_masks = torch.tensor(dev_masks)

test_inputs = torch.tensor(test_inputs)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)

test_df_index = getIndex(test_df)
test_df_inputs = torch.tensor(test_df_inputs)
test_df_masks = torch.tensor(test_df_masks)

In [23]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels)
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

test_df_data = TensorDataset(test_df_index, test_df_inputs, test_df_masks)
test_df_sampler = RandomSampler(test_df_data)
test_df_dataloader = DataLoader(test_df_data, sampler=test_df_sampler, batch_size=batch_size)

# 2. 모델 구현

In [24]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [25]:
model = BertForSequenceClassification.from_pretrained(TRAIN_MODEL, num_labels=8)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [26]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8
                )

total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

#학습

In [27]:
from sklearn.metrics import f1_score

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)


# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [28]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
             
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()


        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy, eval_f1 = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in dev_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
     
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        # tmp_eval_f1 = getF1Score(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        # eval_f1 += tmp_eval_f1
        nb_eval_steps += 1

    print("  Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
    # print("  F1: {0:.2f}".format(eval_f1/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 20 ========
Training...

  Average training loss: 1.33022
  Training epcoh took: 0:00:58

Running Validation...
  Accuracy: 0.54015
  Validation took: 0:00:02

======== Epoch 2 / 20 ========
Training...

  Average training loss: 1.11081
  Training epcoh took: 0:00:58

Running Validation...
  Accuracy: 0.54184
  Validation took: 0:00:02

======== Epoch 3 / 20 ========
Training...

  Average training loss: 0.94392
  Training epcoh took: 0:00:58

Running Validation...
  Accuracy: 0.54749
  Validation took: 0:00:02

======== Epoch 4 / 20 ========
Training...

  Average training loss: 0.77170
  Training epcoh took: 0:00:58

Running Validation...
  Accuracy: 0.51462
  Validation took: 0:00:02

======== Epoch 5 / 20 ========
Training...

  Average training loss: 0.63459
  Training epcoh took: 0:00:58

Running Validation...
  Accuracy: 0.53463
  Validation took: 0:00:02

======== Epoch 6 / 20 ========
Training...

  Average training loss: 0.51007
  Training epcoh took: 0:00

In [29]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


# f1-score parameter
from sklearn.metrics import f1_score
f1_score_avg = []
trues = []
preds = []

In [30]:
#시작 시간 설정
t0 = time.time()
#test_result = test.copy(deep = True)
#test_result = test_result.drop(columns = ['i_dialog', 'i_utterance', 'speaker'])
#test_result['Predicted'] = 'default'

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    #b_index, b_input_ids, b_input_mask = batch
    #b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

   #
    pred_flat = np.argmax(logits, axis=1).flatten()
    trues_flat = label_ids.flatten()
    trues.extend(trues_flat)
    preds.extend(pred_flat)
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
print("  f1 score macro: {0: 4f}".format(f1_score(y_true=trues,y_pred= preds, labels = [0,1,2,3,4,5,6,7], average='macro')))
print("  f1 score micro: {0: 4f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1,2,3,4,5,6,7],average='micro')))
print("  f1 score weighted: {0: 4f}".format(f1_score(y_true=trues, y_pred= preds, labels = [0,1,2,3,4,5,6,7],average='weighted')))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.5559
  f1 score macro:  0.372266
  f1 score micro:  0.555716
  f1 score weighted:  0.550181
Test took: 0:00:04


In [32]:
encoder = LabelEncoder()
classes = emotions
encoder.fit(classes)
classes = encoder.transform(classes)

In [33]:
test_acc = pd.DataFrame(
    {'trues':encoder.classes_[trues]
    ,'preds':encoder.classes_[preds]
    }
  )

In [34]:
acc = (test_acc.trues == test_acc.preds).mean()
acc

0.5557163531114327

In [35]:
acc_idx = test_acc['preds'][test_acc['preds']==test_acc['trues']].index

In [36]:
test_acc.loc[acc_idx].groupby(['preds']).size().reset_index(name = 'count')


,preds,count
0,anger,41
1,disgust,6
2,fear,3
3,joy,184
4,neutral,959
5,non-neutral,187
6,sadness,25
7,surprise,131


In [37]:
test_acc.groupby(['preds']).size().reset_index(name = 'count')


,preds,count
0,anger,79
1,disgust,33
2,fear,29
3,joy,376
4,neutral,1310
5,non-neutral,597
6,sadness,58
7,surprise,282


# test 결과물

In [44]:
tmp_test_dataloader = DataLoader(test_df_data, sampler=test_df_sampler, batch_size=1)
test_result = test_df.copy(deep = True)
test_result = test_result.drop(columns = ['i_dialog', 'i_utterance', 'speaker'])
test_result['Predicted'] = 'default'
classes = [0,1,2,3,4,5,6,7]

encoder = LabelEncoder()
classes = emotions
encoder.fit(classes)
classes = encoder.transform(classes)

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(tmp_test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(tmp_test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_index, b_input_ids, b_input_mask = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    idx = b_index.item()
    test_result['Predicted'][idx] = encoder.classes_[np.argmax(logits)]
    

    nb_eval_steps += 1

print("")
print("Test took: {:}".format(format_time(time.time() - t0)))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


  Batch   100  of  1,623.    Elapsed: 0:00:01.
  Batch   200  of  1,623.    Elapsed: 0:00:02.
  Batch   300  of  1,623.    Elapsed: 0:00:04.
  Batch   400  of  1,623.    Elapsed: 0:00:05.
  Batch   500  of  1,623.    Elapsed: 0:00:06.
  Batch   600  of  1,623.    Elapsed: 0:00:07.
  Batch   700  of  1,623.    Elapsed: 0:00:09.
  Batch   800  of  1,623.    Elapsed: 0:00:10.
  Batch   900  of  1,623.    Elapsed: 0:00:11.
  Batch 1,000  of  1,623.    Elapsed: 0:00:12.
  Batch 1,100  of  1,623.    Elapsed: 0:00:13.
  Batch 1,200  of  1,623.    Elapsed: 0:00:14.
  Batch 1,300  of  1,623.    Elapsed: 0:00:16.
  Batch 1,400  of  1,623.    Elapsed: 0:00:17.
  Batch 1,500  of  1,623.    Elapsed: 0:00:18.
  Batch 1,600  of  1,623.    Elapsed: 0:00:19.

Test took: 0:00:19


In [45]:
test_result['Predicted']

0           neutral
1               joy
2             anger
3           neutral
4       non-neutral
           ...     
1618        neutral
1619            joy
1620        neutral
1621        neutral
1622          anger
Name: Predicted, Length: 1623, dtype: object

In [46]:
test_result

,id,utterance,Predicted
0,0,"Alright, whadyou do with him?",neutral
1,1,Oh! You're awake!,joy
2,2,Then you gotta come clean with Ma! This is not...,anger
3,3,"Yeah, but this is",neutral
4,4,I don't wanna hear it! Now go to my room!,non-neutral
...,...,...,...
1618,1618,Nooo.,neutral
1619,1619,"Hi, Kate!",joy
1620,1620,"Hi, Lauren.",neutral
1621,1621,"Hi, Lauren.",neutral


In [47]:
test_result.drop(labels='utterance', axis="columns", inplace=True)

In [48]:
test_csv = test_result.to_csv('submission_bert_uncase_20.csv', columns=['id', 'Predicted'], index=False)
from google.colab import files

files.download('submission_bert_uncase_20.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>